## Day 24: Electromagnetic Moat

The CPU itself is a large, black building surrounded by a bottomless pit. Enormous metal tubes extend outward from the side of the building at regular intervals and descend down into the void. There's no way to cross, but you need to get inside.

No way, of course, other than building a **bridge** out of the magnetic components strewn about nearby.

### Part One

Each component has two **ports**, one on each end. The ports come in all different types, and only matching types can be connected. You take an inventory of the components by their port types (your puzzle input). Each port is identified by the number of **pins** it uses; more pins mean a stronger connection for your bridge. A 3/7 component, for example, has a type-3 port on one side, and a type-7 port on the other.

Your side of the pit is metallic; a perfect surface to connect a magnetic, **zero-pin port**. Because of this, the first port you use must be of type 0. It doesn't matter what type of port you end with; your goal is just to make the bridge as strong as possible.

The **strength** of a bridge is the sum of the port types in each component. For example, if your bridge is made of components $0/3$, $3/7$, and $7/4$, your bridge has a strength of $0+3 + 3+7 + 7+4 = 24$.

For example, suppose you had the following components:

    0/2
    2/2
    2/3
    3/4
    3/5
    0/1
    10/1
    9/10

With them, you could make the following valid bridges:

    0/1
    0/1--10/1
    0/1--10/1--9/10
    0/2
    0/2--2/3
    0/2--2/3--3/4
    0/2--2/3--3/5
    0/2--2/2
    0/2--2/2--2/3
    0/2--2/2--2/3--3/4
    0/2--2/2--2/3--3/5

(Note how, as shown by 10/1, order of ports within a component doesn't matter. However, you may only use each port on a component once.)

Of these bridges, the strongest one is 0/1--10/1--9/10; it has a strength of $0+1 + 1+10 + 10+9 = 31$.

**What is the strength of the strongest bridge you can make with the components you have available?**

In [33]:
class Component:
    
    def __init__(self, s):
        (self.left, self.right) = [int(_) for _ in s.split('/')]
        
    def rotate(self):
        self.left, self.right = self.right, self.left
        
    def __iter__(self):
        return (_ for _ in [self.left, self.right])
    
    def __str__(self):
        return '[{}, {}]'.format(self.left, self.right)
    
    def __repr__(self):
        return 'Component("{}/{}")'.format(self.left, self.right)
        
# tests

p = Component('2/3')
assert p.left == 2
assert p.right == 3
p.rotate()
assert p.left == 3
assert p.right == 2
assert 2 in p
assert 3 in p
assert 4 not in p
print('ok')

ok


In [57]:
def load_components(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()    
    return [Component(_) for _ in lines]
    
def as_path(path):
    return '--'.join([
        '{}/{}'.format(n.left, n.right)
        for n in path
        ])
    
    
def find_path(path, options):
    global score
    last_component = path[-1]
    port = last_component.right
    new_options = set([o for o in options if port in o])
    if new_options:
        for opt in new_options:
            subset = set(options)
            subset.discard(opt)
            if last_component.right != opt.left:
                opt.rotate()
            find_path(path + [opt], subset)
    else:
        sum_all = sum([n.left+n.right for n in path])
        score.append((sum_all, path))
        return sum_all

def start_find(components):
    start_nodes = [p for p in components if 0 in p]
    for node in start_nodes:
        if node.right == 0:
            node.rotate()        
    for node in start_nodes:
        path = [node]
        options = set(components[:])
        options.discard(node)
        find_path(path, options)

components = load_components('input_test.txt')
score = []
start_find(components)
for w, p in sorted(score, reverse=True):
    print(w, as_path(p))

31 0/1--1/10--10/9
19 0/2--2/2--2/3--3/5
18 0/2--2/2--2/3--3/4
15 0/2--2/3--3/5
14 0/2--2/3--3/4


In [69]:
len(score)

194886

In [58]:
components = load_components('input.txt')
score = []
start_find(components)
for w, p in sorted(score, reverse=True):
    print(w, p)

TypeError: unorderable types: Component() < Component()

In [73]:
local_max = 0
for w, p in score:
    if int(w) > local_max:
        local_max = int(w)
        local_path = p[:]
        
print('Part one:', local_max)

Part one: 1859


### Part two:
    
The bridge you've built isn't long enough; you can't jump the rest of the way.

In the example above, there are two longest bridges:

    0/2--2/2--2/3--3/4
    0/2--2/2--2/3--3/5

Of them, the one which uses the 3/5 component is stronger; its strength is 0+2 + 2+2 + 2+3 + 3+5 = 19.

What is the strength of the longest bridge you can make? If you can make multiple bridges of the longest length, pick the strongest one.

In [82]:
longets_path = max([len(p) for w, p in score])
print('Longest path is {}'.format(longets_path))

max_paths = [(w, p) for w, p in score if len(p) == longets_path]
local_max = 0
for w, p in max_paths:
    if int(w) > local_max:
        local_max = int(w)
        local_path = p[:]
print('Part one:', local_max, as_path(local_path))    
print()
for w, p in max_paths:
    print(w, as_path(p[0:7] + p[-3:]))

Longest path is 35
Part one: 1799 0/50--50/41--41/3--3/10--25/10--49/25--2/49--2/13--43/13--43/43--19/43--36/19--21/36--49/21--49/49--49/32--32/32--32/11--11/6--9/6--39/9--39/50--50/17--17/0--0/36--36/33--33/44--44/22--22/14--14/14--14/40--40/16--16/4--4/24--24/7

1781 0/50--50/41--41/3--3/10--25/10--49/25--2/49--16/4--4/24--24/7
1799 0/50--50/41--41/3--3/10--25/10--49/25--2/49--16/4--4/24--24/7
1799 0/50--50/41--41/3--3/10--25/10--49/25--2/49--16/4--4/24--24/7
1799 0/50--50/41--41/3--3/10--25/10--49/25--49/21--16/4--4/24--24/7
1799 0/50--50/41--41/3--3/10--25/10--49/25--49/21--16/4--4/24--24/7
1781 0/50--50/41--41/3--3/10--25/10--49/25--49/21--16/4--4/24--24/7
1781 0/50--50/41--41/3--3/10--25/10--49/25--49/49--16/4--4/24--24/7
1799 0/50--50/41--41/3--3/10--25/10--49/25--49/49--16/4--4/24--24/7
1799 0/50--50/41--41/3--3/10--25/10--49/25--49/49--16/4--4/24--24/7
1799 0/50--50/41--41/3--3/10--25/10--49/25--49/49--16/4--4/24--24/7
1799 0/50--50/41--41/3--3/10--25/10--49/25--49/49--16/4--4